DATA PREPROCESSING

In [ ]:
#Import Libraries
import pandas as pd
import numpy as np
import os

# Display settings
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 150)

In [ ]:
#User Configuration
# ------------------------------------
# Provide file paths for all input datasets
data_files = [
    "data/nifty.csv",
    "data/global_indices.csv",    #example data files to be replaced with actual file paths
    "data/macro_factors.csv"
]

merge_data = True          # Set False if only one file or no merging needed
merge_on = "Date"          # Common column to merge on
drop_columns = ["Unnamed: 0", "Extra_Notes"]  # Columns to remove if present
date_column = "Date"       # Column name for date parsing (optional)

In [ ]:
#Load & Combine Data
dataframes = []
for f in data_files:
    if os.path.exists(f):
        print(f" Loading file: {f}")
        df_temp = pd.read_csv(f)
        dataframes.append(df_temp)
    else:
        print(f" File not found: {f}")

if not dataframes:
    raise ValueError("No valid data files found. Check file paths.")

if merge_data and len(dataframes) > 1:
    df = dataframes[0]
    for next_df in dataframes[1:]:
        df = pd.merge(df, next_df, on=merge_on, how="inner")
    print(f"\n Successfully merged {len(dataframes)} files on '{merge_on}'.")
else:
    df = dataframes[0]
    print(f"\n Loaded single dataset: {data_files[0]}")

In [ ]:
#Optional: Drop Unwanted Columns
df = df.drop(columns=[col for col in drop_columns if col in df.columns], errors="ignore")
print(f"\n Dropped columns: {[col for col in drop_columns if col in df.columns]}")


In [ ]:
#Basic Data Overview
print("\n📊 DATA OVERVIEW")
print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns")
print("\nColumns:\n", list(df.columns))

# Date parsing (if applicable)
if date_column in df.columns:
    df[date_column] = pd.to_datetime(df[date_column], errors="coerce")
    df = df.sort_values(by=date_column).reset_index(drop=True)

# Show first few rows
print("\nPreview:")
display(df.head())


In [ ]:
#Missing Value Summary
print("\n🔍 MISSING VALUES CHECK")
missing = df.isnull().sum()
missing = missing[missing > 0]

if missing.empty:
    print("No missing values found ✅")
else:
    print("\nColumns with missing values:")
    print(missing.sort_values(ascending=False))

    print("\nPercentage of missing data per column:")
    print((missing / len(df) * 100).round(2))